# Using ResNet50 with Fine-Tuning Strategy

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
# from google.colab import drive 
from matplotlib import pyplot as plt
from tqdm import tqdm


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Data Loader

In [2]:
def data_loader(data_dir, batch_size, shuffle = True): 
  # data augumentation
  transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225],
    ),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees = (-90, 90)),
    # transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1))
  ])

  
  dataset = datasets.ImageFolder(data_dir, transform = transform)
  # classes = dataset.classes

  # Split data into Training and Testing Set
  data_split = torch.utils.data.random_split(dataset, lengths = [0.95, 0.05])
  dataset_train = data_split[0]
  dataset_test = data_split[1]
  
  train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle = shuffle)

  test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle = shuffle)

  return (train_loader, test_loader)




train_loader, test_loader = data_loader('./Data', batch_size = 64)

## Defining Pretrain Model

In [3]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# update output layer to 4 classes
resnet50.fc = nn.Linear(in_features=2048, out_features=4, bias=True)

resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Implementing Fine-Tuning Strategy

In [4]:
# Freeze all layers
for param in resnet50.parameters():
    param.requires_grad = False


for param in resnet50.layer4.parameters():
    param.requires_grad = True

for param in resnet50.fc.parameters():
    param.requires_grad = True


# test code for viewing model parameters
# for name, param in resnet50.named_parameters():
#     if param.requires_grad:
#         print(name)

In [5]:
# Move Model to GPU
resnet50 = resnet50.to(device)

## Hyperparameters

In [6]:
num_epochs = 10
learning_rate = 0.01

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet50.parameters(), lr=learning_rate, weight_decay = 0, momentum = 0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-02.


## Training Model

In [7]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in loop:  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # progress bar descriptor
        loop.set_description(f'Epoch [{epoch}/{num_epochs}]')
        loop.set_postfix(loss = loss.item())

        # if (i % 100 == 0):
        #     print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        #            .format(epoch+1, num_epochs, i, total_step, loss.item()))
            
    # Validation
    resnet50.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        loop_val = tqdm(train_loader)
        for images, labels in loop_val:
            images = images.to(device)
            labels = labels.to(device)
            outputs = resnet50(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
            loop_val.set_description('Validation Accuracy')
            loop_val.set_postfix(Accuracy = correct / total)
        # print('Accuracy of the network on the {} validation images: {} %'.format(200, 100 * correct / total)) 
    
    scheduler.step()
    resnet50.train()

    # save
    torch.save(resnet50, f"./trained_models/resnet50_dog_emotion_{epoch+1}.pt")

Validation Accuracy: 100%|██████████| 296/296 [06:00<00:00,  1.22s/it, Accuracy=0.586]


Adjusting learning rate of group 0 to 1.0000e-02.


Validation Accuracy: 100%|██████████| 296/296 [05:54<00:00,  1.20s/it, Accuracy=0.613]


Adjusting learning rate of group 0 to 1.0000e-03.


Validation Accuracy: 100%|██████████| 296/296 [05:57<00:00,  1.21s/it, Accuracy=0.631]


...

Adjusting learning rate of group 0 to 1.0000e-06.


Validation Accuracy: 100%|██████████| 296/296 [05:55<00:00,  1.20s/it, Accuracy=0.633]


Adjusting learning rate of group 0 to 1.0000e-06.


Validation Accuracy: 100%|██████████| 296/296 [05:57<00:00,  1.21s/it, Accuracy=0.638]


Adjusting learning rate of group 0 to 1.0000e-07.


In [9]:
resnet50.eval()
correct = 0
total = 0
with torch.no_grad():
    loop_val = tqdm(test_loader)
    for images, labels in loop_val:
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs
        loop_val.set_description('Test Accuracy')
        loop_val.set_postfix(Accuracy = correct / total)
    # print('Accuracy of the network on the {} validation images: {} %'.format(200, 100 * correct / total)) 

resnet50.train()

Test Accuracy: 100%|██████████| 16/16 [00:31<00:00,  1.97s/it, Accuracy=0.585]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 